# First Init

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 10)
df = pd.read_csv('https://query.data.world/s/rkwkja2ibruiaaoevtkpba55jjshys')

usable_df = df[['prices.amountMax', 'prices.condition', 'brand',  'name']]
usable_df.drop(usable_df[(usable_df['prices.condition'] == 'Used')].index, \
                          inplace=True)

# current usd to idr rate
usable_df[['prices.amountMax']] = usable_df[['prices.amountMax']].multiply(14251.65)
usable_df['prices.amountMax'] = usable_df['prices.amountMax'].round(decimals=2)

# set new column
usable_df['overpriced'] = 0

usable_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

,prices.amountMax,prices.condition,brand,name,overpriced
0,1496280.73,New,Sanus,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,0
1,983363.85,New,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,0
2,983363.85,New,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,0
3,997472.98,New,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,0
4,954718.03,New,Boytone,Boytone - 2500W 2.1-Ch. Home Theater System - ...,0
...,...,...,...,...,...
7244,1139989.48,New,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,0
7245,979800.94,New,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,0
7246,1011724.63,New,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,0
7247,1025976.28,New,Lowepro,Lowepro - Slingshot Edge 250 AW Camera Backpac...,0


In [ ]:
array_item = ['Sony - BC-TRX Battery Charger - Black', \
              'Sony HTST9 Soundbar with Wireless Subwoofer Bluetooth and Google Cast', \
              '15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)', \
              'iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging', \
              'GoSafe S30 1080p Dash Cam', \
              'Sideclick - Universal Remote Attachment for Roku® Streaming Players - Black', \
              'Samsung - Adaptive Fast Charging Wall Charger - White', \
              'SanDisk Ultra II 1TB SATA III SSD - 2.5-Inch 7mm Height Solid State Drive - SDSSDHII-1T00-G25', \
              'Olympus TG-5 Waterproof Camera with 3-Inch LCD', \
              'Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3000MHz C15 Desktop Memory Kit - Black (CMK16GX4M2B3000C15)']

In [ ]:
test_df = pd.DataFrame(columns=['prices.amountMax', 'prices.condition', 'brand', 'name', 'overpriced'])

for item in array_item:
  # assign to temporary dataframe
  temp_df = usable_df[usable_df['name'] == item]
  
  # look for standard deviation
  price_std = temp_df['prices.amountMax'].std()

  # assign underprice data
  for i in range(1, 16):
    new_min = temp_df['prices.amountMax'].min() - price_std
    if new_min > 0:
      temp_df.loc[len(temp_df.index)] = [new_min, 'New', temp_df['brand'].iloc[0], item, 0]
    else:
      break

  # assign overprice data
  for i in range(1, 16):
    new_max = temp_df['prices.amountMax'].max() + price_std
    temp_df.loc[len(temp_df.index)] = [new_max, 'New', temp_df['brand'].iloc[0], item, 1]

  test_df = pd.concat([test_df, temp_df], ignore_index=True)

test_df['prices.amountMax'] = test_df['prices.amountMax'].round(decimals=2)
test_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,prices.amountMax,prices.condition,brand,name,overpriced
0,327645.43,New,Sony,Sony - BC-TRX Battery Charger - Black,0
1,413155.33,New,Sony,Sony - BC-TRX Battery Charger - Black,0
2,555671.83,New,Sony,Sony - BC-TRX Battery Charger - Black,0
3,384652.03,New,Sony,Sony - BC-TRX Battery Charger - Black,0
4,472442.20,New,Sony,Sony - BC-TRX Battery Charger - Black,0
...,...,...,...,...,...
361,5730642.82,New,Corsair,Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3...,1
362,5999200.68,New,Corsair,Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3...,1
363,6267758.53,New,Corsair,Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3...,1
364,6536316.38,New,Corsair,Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3...,1


In [ ]:
test_df['prices.condition'] = "NEW"

encoded_df = pd.get_dummies(test_df)

encoded_df

,prices.amountMax,prices.condition_NEW,brand_Apple,brand_Corsair,brand_Olympus,brand_Papago,brand_Samsung,brand_SanDisk,brand_Sideclick,brand_Sony,brand_Sound Design,"name_15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",name_Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3000MHz C15 Desktop Memory Kit - Black (CMK16GX4M2B3000C15),name_GoSafe S30 1080p Dash Cam,name_Olympus TG-5 Waterproof Camera with 3-Inch LCD,name_Samsung - Adaptive Fast Charging Wall Charger - White,name_SanDisk Ultra II 1TB SATA III SSD - 2.5-Inch 7mm Height Solid State Drive - SDSSDHII-1T00-G25,name_Sideclick - Universal Remote Attachment for Roku® Streaming Players - Black,name_Sony - BC-TRX Battery Charger - Black,name_Sony HTST9 Soundbar with Wireless Subwoofer Bluetooth and Google Cast,name_iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,overpriced_0,overpriced_1
0,327645.43,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
1,413155.33,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
2,555671.83,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
3,384652.03,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
4,472442.20,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,5730642.82,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
362,5999200.68,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
363,6267758.53,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
364,6536316.38,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


# Modeling

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
x_data = encoded_df.iloc[:, :-2]
y_data = encoded_df.iloc[:, -1]

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.4, random_state=42)

xgb_model = xgb.XGBClassifier(learning_rate=0.001,
                            max_depth = 1, 
                            n_estimators = 100,
                            scale_pos_weight=5)

xgb_model.fit(x_train, y_train)

print("Accuracy on training set: {:.3f}".format(xgb_model.score(x_train, y_train)))
print("Accuracy on validation set: {:.3f}".format(xgb_model.score(x_test, y_test)))

Accuracy on training set: 0.553
Accuracy on validation set: 0.571


# Export Model

In [ ]:
xgb_model.save_model('overprice.bin')
print("Saved model to: overprice.bin")


Saved model to: overprice.bin
